In [3]:
import numpy as np

In [4]:
# 加载训练集、测试集数据（行为用户，列为物品，train[u][i]=rate，test同上）
def load_movielens(path='../ml-100k', k="1"):
    # get movie titles
    movies = {}
    prefs_shape = []
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = title
    # load data
    for line in open(path + '/u.info', encoding='latin-1'):
        prefs_shape.append(int(line.split(' ')[0:1][0]))
    train = np.zeros(shape=prefs_shape[0:2], dtype=np.int)
    test = np.zeros(shape=prefs_shape[0:2], dtype=np.int)
    for line in open(path + '/u' + k + '.base', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        train[int(user)-1][int(movieid)-1] = int(rating)
    for line in open(path + '/u' + k + '.test', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        test[int(user)-1][int(movieid)-1] = int(rating)
    return train, test
train, test = load_movielens()

In [5]:
def sgd(data_matrix, k, alpha, lam, max_cycles):
    """使用梯度下降法进行矩阵分解。

    Args:
    - data_matrix: mat, 用户物品矩阵
    - k: int, 分解矩阵的参数
    - alpha: float, 学习率
    - lam: float, 正则化参数
    - max_cycles: int, 最大迭代次数

    Returns:
    p,q: mat, 分解后的矩阵
    """
    m, n = np.shape(data_matrix)
    # initiate p & q
    p = np.mat(np.random.random((m, k)))
    q = np.mat(np.random.random((k, n)))

    # start training
    for step in range(max_cycles):
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = data_matrix[i, j]
                    for r in range(k):
                        error = error - p[i, r] * q[r, j]
                    for r in range(k):
                        p[i, r] = p[i, r] + alpha * (2 * error * q[r, j] - lam * p[i, r])
                        q[r, j] = q[r, j] + alpha * (2 * error * p[i, r] - lam * q[r, j])

        loss = 0.0
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = 0.0
                    for r in range(k):
                        error = error + p[i, r] * q[r, j]
                    # calculate loss function
                    loss = (data_matrix[i, j] - error) * (data_matrix[i, j] - error)
                    for r in range(k):
                        loss = loss + lam * (p[i, r] * p[i, r] + q[r, j] * q[r, j]) / 2

        if loss < 0.001:
            break
        if step % 1 == 0:
            print("\titer: %d, loss: %f" % (step, loss))
    return p, q
p, q = sgd(train, 2, 1e-4, 0.01, 10)

	iter: 0, loss: 4.205896
	iter: 1, loss: 3.848058
	iter: 2, loss: 3.495190
	iter: 3, loss: 3.153371
	iter: 4, loss: 2.828353
	iter: 5, loss: 2.525054
	iter: 6, loss: 2.247159
	iter: 7, loss: 1.996888
	iter: 8, loss: 1.774969
	iter: 9, loss: 1.580812


In [13]:
# 基于矩阵分解为用户 user 推荐物品
def prediction(data_matrix, p, q, user):
    """为用户未互动的项打分

    Args:
    - data_matrix: mat, 原始用户物品矩阵
    - p: mat, 分解后的矩阵p
    - q: mat, 分解后的矩阵q
    - user: int, 用户的id

    Returns:
    - predict: list, 推荐列表
    """
    n = np.shape(data_matrix)[1]
    predict = {}
    for j in range(n):
        if data_matrix[user, j] == 0:
            predict[j] = (p[user,] * q[:, j])[0, 0]

    # 按照打分从大到小排序
    return sorted(predict.items(), key=lambda d: d[1], reverse=True)

def top_k(predict, n):
    """为用户推荐前 n 个物品

    Args:
    - predict: list, 排好序的物品列表
    - k: int, 推荐的物品个数

    :return: top_recom, list, top n 个物品
    """
    top_recom = []
    len_result = len(predict)
    if n >= len_result:
        top_recom = predict
    else:
        for i in range(n):
            top_recom.append(predict[i])
    return top_recom

In [14]:
def recall(train, test, p, q):
    hit = 0
    all = 0
    for u in range(train.shape[0]):
        tu = test[u]
        predict = prediction(train, p, q, u)
        T = (tu>0).sum()
        pre_tu = top_k(predict, 5)
        for item, _ in pre_tu:
            if item in tu:
                hit += 1
        all += T        
    return hit / (all * 1.0)
recall(train, test, p, q)

0.0172

In [15]:
def precision(train, test, p, q):
    hit = 0
    all = 0
    for u in range(train.shape[0]):
        tu = test[u]
        predict = prediction(train, p, q, u)
        pre_tu = top_k(predict, 5)
        for item, _ in pre_tu:
            if item in tu:
                hit += 1
        all += 5
    return hit / (all * 1.0)
precision(train, test, p, q)

0.07295864262990456

In [17]:
def coverage(train, test, p, q):
    recommend_items = set()
    all_items = set()
    for u in range(train.shape[0]):
        for i in range(train.shape[1]):
            all_items.add(i)
        predict = prediction(train, p, q, u)
        pre_tu = top_k(predict, 5)
        for item, _ in pre_tu:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)
coverage(train, test, p, q)

0.028537455410225922

In [19]:
def popularity(train, test, p, q):
    import math
    item_popularity = dict()
    for i in range(train.shape[1]):
        if i not in item_popularity:
            item_popularity.setdefault(i, 0)
        item_popularity[i] += 1
    ret = 0
    n = 0
    for u in range(train.shape[0]):
        predict = prediction(train, p, q, u)
        pre_tu = top_k(predict, 5)
        for item, _ in pre_tu:
            ret += math.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret
popularity(train, test, p, q)

0.6931471805599841